### Conditional GAN

CelebA comes with a list of 40 binary attributes. Theses attributes include gender. We can extract that as the class label and we have a binary class label.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import os

In [2]:
def discriminator():
    img = layers.Input(shape=[64,64,3])
    label = layers.Input(shape=[2,]) #for two classes
    y = layers.Dense(64*64)(label)
    x = layers.LeakyReLU(0.2)(x)
    y = layers.Reshape((64,64,1))(y)
    
    x = layers.concatenate([img, y])
    
    x = layers.Conv2D(filters=128, kernel_size=5, strides=2, padding='same')(x)
    for filter_size in [256, 512,1024]:
        x = layers.Conv2D(filters=filter_size, kernel_size=5, strides=2, padding='same')(x)
        x = layers.LeakyReLU(0.2)(x)
        x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)
    return Model(inputs = [img, label], outputs = x)
    

In [3]:
def generator(noise_dim):
    z = layers.Input(shape=noise_dim)
    x = layers.Dense(units=7*7*256)(z)
    x = layers.Reshape((7,7,256))(x)
    
    label = layers.Input(shape=[2,]) #for two classes
    y = layers.Dense(7*7)(label)
    x = layers.LeakyReLU(0.2)(x)
    y = layers.Reshape((7,7,1))(y)
    
    x = layers.concatenate([x, y])

    for filter_size in [256,128, 64, num_channels]:
        x = layers.Conv2DTranspose(filters=filter_size, kernel_size=5, strides=2, padding='same')(x)
        x = layers.LeakyReLU(0.2)(x)
        x = layers.BatchNormalization()(x)
    return Model(inputs = [z, label], outputs=x)

In [4]:
@tf.function
def train_step(images, labels):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = G(noise, training=True)
        fake_labels = np.random.randint(0, 2, BATCH_SIZE)
        
        real_output = D([images,labels], training=True)
        fake_output = D([generated_images,fake_labels], training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, G.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, D.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, G.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, D.trainable_variables))

Conditional GAN with Resnet 50

In [5]:
celeba_path = "C:\\Users\\tghosh\\Syncplicity\\Work\\bpbonline\\Chapter_13\\celebfaces"
input_path = celeba_path#".\\YOUR_PATH\\celebfaces"
def load_real_samples():
    
    # Loading Dataset
    data = np.load(os.path.join(input_path, 'img_celeba.npz'))
    train_x = data['arr_0']    
    print(data.keys())
    train_x = train_x.astype('float32')
    train_x = (train_x - 127.5) / 127.5
    return train_x

# Loading Dataset
dataset = load_real_samples()

KeyError: 'arr_1 is not a file in the archive'

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        print('starting epoch {}'.format(epoch))
        
        for image_batch in train_dataset:   
            if image_batch.shape[0]==BATCH_SIZE:
                train_step(image_batch)
        print ('Time for epoch {} is {} sec, loss {}'.format(epoch + 1, time.time()-start, total_loss_tracker.result()))
        if epoch % 5 == 0:
            save_images(epoch)

In [6]:
tfds.image.CelebA

NameError: name 'tfds' is not defined

In [7]:
from tfds.image import  CelebA

ModuleNotFoundError: No module named 'tfds'